In [15]:
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import numpy as np

## Cleaning the raw data

In [16]:
# Create dictionary from df for faster lookup
postcodes_df = pd.read_csv('ukpostcodes.csv')

### Import the rows of interest using Dask for better performance

In [17]:
# Specify file path of raw data
file_path = 'pp-complete.csv'

# Define column names
column_names = ['transaction_id','price', 'transfer_date', 'postcode', 'property_type', 'new_build_flag', 'duration', 'primary_address', 'secondary_address', 'street', 'locality', 'city', 'district', 'county', 'ppd_category', 'record_status']

# Specify data types for each column
dtypes = dict.fromkeys(column_names, 'string')
dtypes.update({'price': 'int64'})

# Specify datetime columns for parsing
parse_dates = ['transfer_date']

# Define columns we don't want
usecols = [col for col in column_names if col != 'transaction_id']

# Read CSV using Dask with specified data types and parse_dates
ddf = dd.read_csv(file_path, header=None, names=column_names, dtype=dtypes, parse_dates=parse_dates,usecols=usecols)

# Compute to get a Pandas DataFrame
raw_data_df = ddf.compute()


### Drop rows with no postcode, properties of type 'Other

In [18]:
# Drop rows with no postcode
clean_data_df = raw_data_df.dropna(subset=['postcode'])

#Exclude property type Other (O)
clean_data_df = clean_data_df.loc[clean_data_df['property_type']!='O']

# Sort by Date
clean_data_df.sort_values(by=['transfer_date'], inplace=True, ignore_index=True)

### Add postcode lat and long

In [19]:
# Read the postcode CSV file
postcode_df = pd.read_csv('ukpostcodes.csv')

# Convert df to dict for faster lookup
postcodes = dict()
for (postcode, latitude, longitude) in postcodes_df[['postcode', 'latitude', 'longitude']].values:postcodes[postcode] = [latitude, longitude]

# Apply the mapping to create the new column
clean_data_df['postcode_lat_long'] = clean_data_df['postcode'].map(postcodes)

### Add postcode sectors

In [20]:
clean_data_df['postcode_sector'] =  clean_data_df['postcode'].apply(lambda x: x[:x.find(' ')+2])

### Add region column as defined by GeoJSON files

In [21]:
# Load regions by postcodes
regions_df = pd.read_csv('postcode_areas.csv', usecols=['postcode_prefix', 'region'])

# Create a dictionary from the DataFrame
postcode_region_dict = dict(zip(regions_df['postcode_prefix'], regions_df['region']))

# Add postcode prefix column to main df
clean_data_df['postcode_prefix'] = clean_data_df['postcode'].str.extract(r'^([A-Z]+)')

# Map the postcode prefix to the region using the dictionary
clean_data_df['region'] = clean_data_df['postcode_prefix'].map(postcode_region_dict)

# Drop the intermediate 'postcode_prefix' column
clean_data_df.drop(columns=['postcode_prefix'], inplace=True)

#Exclude property type Other (O)
clean_data_df = clean_data_df.loc[clean_data_df['region']!='Scotland']

### Add year column

In [22]:
# Extract the year from 'transfer_date'
clean_data_df['year'] = clean_data_df['transfer_date'].dt.year

# Extract the month from 'transfer_date'
clean_data_df['month'] = clean_data_df['transfer_date'].dt.month

## Create GeoJSON files

### Convert Shapefile to GeoJSON

In [8]:
# Create GeoJSON file for all sectors using Shapefil
myshpfile = gpd.read_file('shapefiles/Sectors.shp')
myshpfile.to_file('geo_json/all_sectors.geojson', driver='GeoJSON')

### Create discrete GeoJSON files for each region

In [10]:
# Load the postcode prefixes CSV
postcode_prefixes_df = pd.read_csv('postcode_areas.csv')

# Load the GeoJSON file containing postcode sector data
geojson_path = 'geo_json/all_sectors.geojson'
postcode_sectors_gdf = gpd.read_file(geojson_path)

# Create an empty dictionary to store GeoDataFrames for each region
region_geojson_dict = {}

# Extract postcode prefix from 'name' property
postcode_sectors_gdf['postcode_prefix'] = pd.Series(postcode_sectors_gdf['name']).str.extract(r'^([A-Z]+)')

# Merge with postcode prefixes DataFrame to get regions
merged_df = pd.merge(postcode_sectors_gdf, postcode_prefixes_df, how='left', on='postcode_prefix')

# Create a dictionary to store GeoDataFrames for each region
region_geojson_dict = {}

# Iterate over unique regions
for region in merged_df['region'].unique():
    # Filter DataFrame for the current region
    region_df = merged_df[merged_df['region'] == region].copy()
    
    # Create a GeoDataFrame for the current region
    region_gdf = gpd.GeoDataFrame(region_df, geometry='geometry', crs=postcode_sectors_gdf.crs)
    
    # Store the GeoDataFrame in the dictionary under the region key
    region_geojson_dict[region] = region_gdf

# Now, region_geojson_dict contains GeoDataFrames for each region

for region, region_gdf in region_geojson_dict.items():
    region_geojson_path = f'geo_json/regions/{region}_postcode_sectors.geojson'
    region_gdf.to_file(region_geojson_path, driver='GeoJSON')

## Process Data

### Calulate average price for each postcode by year and export processed data

In [26]:
# Group by region, postcode_sector, and year, then calculate the rounded average price and count the number of sales
grouped_df = clean_data_df.groupby(['region', 'postcode_sector', 'year']).agg({'price': ['mean', 'count']}).reset_index()
grouped_df.columns = ['region', 'postcode_sector', 'year','avg_price', 'volume']

# Round the average price to the nearest thousand and convert to integers
grouped_df['avg_price'] = grouped_df['avg_price'].round(-3).astype(int)

# Loop through unique years and save CSV for each year
for year in grouped_df['year'].unique():
    # Filter DataFrame for the current year
    year_df = grouped_df[grouped_df['year'] == year]
    
    # Create the CSV file path for the current year
    csv_file_path = (f'processed_data/average_price_by_year/region_data_{year}.csv')
    
    # Save the filtered DataFrame to CSV
    year_df.to_csv(csv_file_path, index=False)

In [26]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235870 entries, 0 to 235869
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   region           235870 non-null  object
 1   postcode_sector  235870 non-null  object
 2   year             235870 non-null  int64 
 3   avg_price        235870 non-null  int32 
 4   volume           235870 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 8.1+ MB


### Calculate average price for each region and export processed data

In [23]:
# Group by year, region, property_type, and calculate rounded average price
region_grouped_df = clean_data_df.groupby(['year', 'region', 'property_type']).agg(avg_price=('price', 'mean')).round({'avg_price': -3}).astype({'avg_price': int}).reset_index()

# Calculate sales volume for each group
region_grouped_df['volume'] = clean_data_df.groupby(['year', 'region', 'property_type']).size().reset_index(name='volume')['volume']

# Save the grouped data to a single CSV file
region_grouped_df.to_csv('processed_data/region_avg_price/region_avg_prices.csv', index=False)

### Calculate the percentage delta of average price (year on year)

In [ ]:
# Group by region and postcode_sector, then calculate the percentage delta
delta_df = grouped_df.copy()  # Create a new DataFrame to store the results
delta_df['delta'] = grouped_df.groupby(['region', 'postcode_sector'])['avg_price'].pct_change() * 100

# Set the percentage change for the first year to 0
delta_df.loc[delta_df['year'] == delta_df['year'].min(), 'delta'] = 0

# Drop any rows with a null delta
delta_df = delta_df.dropna(subset=['delta'])

# Round values to the nearest integer
delta_df['delta'] = delta_df['delta'].round().astype(int)

In [43]:
# Get list of years from df
unique_years = delta_df['year'].unique()

# Iterate over each year and create a CSV file
for year in unique_years:
    # Filter rows for the current year
    year_df = delta_df[delta_df['year'] == year]

    # Create a CSV file for the current year
    csv_file_path = f'processed_data/avg_price_delta/avg_price_delta_{year}.csv'
    year_df.to_csv(csv_file_path, index=False)

In [13]:
clean_data_df.tail()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status,postcode_lat_long,year,month
28141180,315000,2023-09-29,LE11 3NF,T,N,F,49,<NA>,KINGFISHER WAY,<NA>,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,B,A,"[52.765639, -1.217486]",2023,9
28141181,85000,2023-09-29,DL14 6SZ,T,N,F,8,<NA>,WEST VIEW,SOUTH CHURCH,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A,"[54.647078, -1.662091]",2023,9
28141182,350000,2023-09-29,HP18 0FU,T,N,F,5,<NA>,ALMA STREET,<NA>,AYLESBURY,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,B,A,"[51.835956, -0.85731]",2023,9
28141183,465000,2023-09-29,E8 3QN,F,N,L,212A,FLAT 3,RICHMOND ROAD,<NA>,LONDON,HACKNEY,GREATER LONDON,A,A,"[51.543252, -0.06186]",2023,9
28141184,125000,2023-09-29,SE14 6DQ,F,N,L,"GROUND FLOOR FLAT, 59",<NA>,CHILDERIC ROAD,<NA>,LONDON,LEWISHAM,GREATER LONDON,A,A,"[51.477684, -0.03846]",2023,9


### Create volume by month for each year

In [32]:
# Group by region, postcode_sector, year, and month, then calculate the count of sales
volume_df = clean_data_df.groupby(['region', 'postcode_sector', 'year', 'month']).agg({'price': 'count'}).reset_index()

# Rename the columns for clarity
volume_df.columns = ['region', 'postcode_sector', 'year', 'month', 'volume']

# Loop through unique years
for year in volume_df['year'].unique():
    # Filter DataFrame for the current year
    year_df = volume_df[volume_df['year'] == year]
    
    # Group by region and month, then calculate the total volume
    total_volume_df = year_df.groupby(['region', 'month']).agg({'volume': 'sum'}).reset_index()
    
    # Create the CSV file path for the current year and save the DataFrame
    csv_file_path = f'processed_data/volume_by_year/region_total_volume_{year}.csv'
    total_volume_df.to_csv(csv_file_path, index=False)